In [1]:
import pandas as pd
import numpy as np
import time
pd.options.display.max_columns
pd.options.display.max_rows
import matplotlib.pyplot as plt
import calendar
%matplotlib inline
plt.style.use('bmh')
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))
#import needed libraries & float format definition
import unicodecsv
import seaborn as sns
import datetime as dt
from scipy import stats
from scipy.stats import norm
pd.options.display.float_format = '{:.0f}'.format

In [2]:
%time df = pd.read_pickle('OTP_11.17.19.pkl') #import data
display(df.shape)
display(df.head())
display(df.info())

Wall time: 40.3 s


(12771253, 60)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,OP_CARRIER_FL_NUM,...,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,Unnamed: 59
0,2018,4,10,1,1,2018-10-01,WN,19393,WN,802,...,7,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2018,4,10,1,1,2018-10-01,WN,19393,WN,3744,...,7,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,2018,4,10,1,1,2018-10-01,WN,19393,WN,1019,...,3,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,2018,4,10,1,1,2018-10-01,WN,19393,WN,1499,...,3,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,2018,4,10,1,1,2018-10-01,WN,19393,WN,3635,...,3,nan,nan,nan,nan,nan,nan,nan,nan,nan


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12771253 entries, 0 to 12771252
Data columns (total 60 columns):
YEAR                     int64
QUARTER                  int64
MONTH                    int64
DAY_OF_MONTH             int64
DAY_OF_WEEK              int64
FL_DATE                  object
OP_UNIQUE_CARRIER        object
OP_CARRIER_AIRLINE_ID    int64
OP_CARRIER               object
OP_CARRIER_FL_NUM        int64
ORIGIN_AIRPORT_ID        int64
ORIGIN_AIRPORT_SEQ_ID    int64
ORIGIN_CITY_MARKET_ID    int64
ORIGIN                   object
ORIGIN_CITY_NAME         object
ORIGIN_STATE_ABR         object
ORIGIN_STATE_NM          object
DEST_AIRPORT_ID          int64
DEST_AIRPORT_SEQ_ID      int64
DEST_CITY_MARKET_ID      int64
DEST                     object
DEST_CITY_NAME           object
DEST_STATE_ABR           object
DEST_STATE_NM            object
CRS_DEP_TIME             int64
DEP_TIME                 float64
DEP_DELAY                float64
DEP_DELAY_NEW            float64


None

In [3]:
#Remove white space from columns and check columns
df.columns = df.columns.str.strip()
df.YEAR.unique()

array([2018, 2019], dtype=int64)

In [4]:
#drop empty column and add Month_year column to check import data set
df = df[df.columns.drop('Unnamed: 59')] # drop unnamed columns
##
%time df['Monthd'] = df['MONTH'].apply(lambda x: calendar.month_abbr[x]) # Create a column with month description in text

#Larger data sets (> 150rows), use below
%time df['MY'] = df['Monthd'].astype(str) + '_' + df['YEAR'].astype(str) # combine new colum with YEAR column

df.MY.unique() # check to make sure all months are imported into dataframe

Wall time: 1min 43s
Wall time: 19.1 s


array(['Oct_2018', 'Nov_2018', 'Dec_2018', 'Jan_2019', 'Feb_2019',
       'Mar_2019', 'Apr_2019', 'May_2019', 'Jun_2019', 'Jul_2019',
       'Jan_2018', 'Aug_2019', 'Sep_2019', 'Feb_2018', 'Mar_2018',
       'Apr_2018', 'May_2018', 'Jun_2018', 'Jul_2018', 'Aug_2018',
       'Sep_2018'], dtype=object)

In [5]:
# check to look at NANs
%time null_columns=df.columns[df.isnull().any()]

#null_columns
%time df[null_columns].isnull().sum()

Wall time: 28.8 s
Wall time: 9.8 s


DEP_TIME                 227753
DEP_DELAY                232693
DEP_DELAY_NEW            232693
DEP_DEL15                232693
DEP_DELAY_GROUP          232693
TAXI_OUT                 234632
WHEELS_OFF               234631
WHEELS_ON                240919
TAXI_IN                  240919
ARR_TIME                 240917
ARR_DELAY                271971
ARR_DELAY_NEW            271971
ARR_DEL15                271971
ARR_DELAY_GROUP          271971
CANCELLATION_CODE      12535155
CRS_ELAPSED_TIME            145
ACTUAL_ELAPSED_TIME      269373
AIR_TIME                 269373
CARRIER_DELAY          10346932
WEATHER_DELAY          10346932
NAS_DELAY              10346932
SECURITY_DELAY         10346932
LATE_AIRCRAFT_DELAY    10346932
FIRST_DEP_TIME         12684061
TOTAL_ADD_GTIME        12684063
LONGEST_ADD_GTIME      12684063
dtype: int64

In [6]:
#Percentage of missing values in each column within the dataset
%time percent_missing = round(df.isnull().mean()*100,2) 
%time percent_missing

Wall time: 17.6 s
Wall time: 0 ns


YEAR                     0
QUARTER                  0
MONTH                    0
DAY_OF_MONTH             0
DAY_OF_WEEK              0
FL_DATE                  0
OP_UNIQUE_CARRIER        0
OP_CARRIER_AIRLINE_ID    0
OP_CARRIER               0
OP_CARRIER_FL_NUM        0
ORIGIN_AIRPORT_ID        0
ORIGIN_AIRPORT_SEQ_ID    0
ORIGIN_CITY_MARKET_ID    0
ORIGIN                   0
ORIGIN_CITY_NAME         0
ORIGIN_STATE_ABR         0
ORIGIN_STATE_NM          0
DEST_AIRPORT_ID          0
DEST_AIRPORT_SEQ_ID      0
DEST_CITY_MARKET_ID      0
DEST                     0
DEST_CITY_NAME           0
DEST_STATE_ABR           0
DEST_STATE_NM            0
CRS_DEP_TIME             0
DEP_TIME                 2
DEP_DELAY                2
DEP_DELAY_NEW            2
DEP_DEL15                2
DEP_DELAY_GROUP          2
                        ..
TAXI_OUT                 2
WHEELS_OFF               2
WHEELS_ON                2
TAXI_IN                  2
CRS_ARR_TIME             0
ARR_TIME                 2
A

In [7]:
%time  df_original = df.copy() #created a copy of original df with all variables prior to reducing columns based on missing values

Wall time: 6.45 s


In [8]:
#Calculate the percent of values in a column with missing values. 
#From those columns, filter out the features with more than 80% NULL values and then drop those columns from the DataFrame.
%time pct_null = df.isnull().sum() / len(df)
%time missing_features = pct_null[pct_null > 0.99].index
%time df.drop(missing_features, axis=1, inplace=True)

Wall time: 26.4 s
Wall time: 1.98 ms
Wall time: 9.21 s


In [9]:
#Percentage of missing values in each column within the dataset
%time percent_missing2 = round(df.isnull().mean()*100,2) 
display(df.shape)
%time percent_missing2

Wall time: 18.3 s


(12771253, 58)

Wall time: 0 ns


YEAR                     0
QUARTER                  0
MONTH                    0
DAY_OF_MONTH             0
DAY_OF_WEEK              0
FL_DATE                  0
OP_UNIQUE_CARRIER        0
OP_CARRIER_AIRLINE_ID    0
OP_CARRIER               0
OP_CARRIER_FL_NUM        0
ORIGIN_AIRPORT_ID        0
ORIGIN_AIRPORT_SEQ_ID    0
ORIGIN_CITY_MARKET_ID    0
ORIGIN                   0
ORIGIN_CITY_NAME         0
ORIGIN_STATE_ABR         0
ORIGIN_STATE_NM          0
DEST_AIRPORT_ID          0
DEST_AIRPORT_SEQ_ID      0
DEST_CITY_MARKET_ID      0
DEST                     0
DEST_CITY_NAME           0
DEST_STATE_ABR           0
DEST_STATE_NM            0
CRS_DEP_TIME             0
DEP_TIME                 2
DEP_DELAY                2
DEP_DELAY_NEW            2
DEP_DEL15                2
DEP_DELAY_GROUP          2
DEP_TIME_BLK             0
TAXI_OUT                 2
WHEELS_OFF               2
WHEELS_ON                2
TAXI_IN                  2
CRS_ARR_TIME             0
ARR_TIME                 2
A

In [10]:
round(df.isnull().mean()*100,2) 

YEAR                     0
QUARTER                  0
MONTH                    0
DAY_OF_MONTH             0
DAY_OF_WEEK              0
FL_DATE                  0
OP_UNIQUE_CARRIER        0
OP_CARRIER_AIRLINE_ID    0
OP_CARRIER               0
OP_CARRIER_FL_NUM        0
ORIGIN_AIRPORT_ID        0
ORIGIN_AIRPORT_SEQ_ID    0
ORIGIN_CITY_MARKET_ID    0
ORIGIN                   0
ORIGIN_CITY_NAME         0
ORIGIN_STATE_ABR         0
ORIGIN_STATE_NM          0
DEST_AIRPORT_ID          0
DEST_AIRPORT_SEQ_ID      0
DEST_CITY_MARKET_ID      0
DEST                     0
DEST_CITY_NAME           0
DEST_STATE_ABR           0
DEST_STATE_NM            0
CRS_DEP_TIME             0
DEP_TIME                 2
DEP_DELAY                2
DEP_DELAY_NEW            2
DEP_DEL15                2
DEP_DELAY_GROUP          2
DEP_TIME_BLK             0
TAXI_OUT                 2
WHEELS_OFF               2
WHEELS_ON                2
TAXI_IN                  2
CRS_ARR_TIME             0
ARR_TIME                 2
A

In [11]:
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,OP_CARRIER_FL_NUM,...,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Monthd,MY
0,2018,4,10,1,1,2018-10-01,WN,19393,WN,802,...,1,1670,7,nan,nan,nan,nan,nan,Oct,Oct_2018
1,2018,4,10,1,1,2018-10-01,WN,19393,WN,3744,...,1,1670,7,nan,nan,nan,nan,nan,Oct,Oct_2018
2,2018,4,10,1,1,2018-10-01,WN,19393,WN,1019,...,1,580,3,nan,nan,nan,nan,nan,Oct,Oct_2018
3,2018,4,10,1,1,2018-10-01,WN,19393,WN,1499,...,1,580,3,nan,nan,nan,nan,nan,Oct,Oct_2018
4,2018,4,10,1,1,2018-10-01,WN,19393,WN,3635,...,1,580,3,nan,nan,nan,nan,nan,Oct,Oct_2018


In [12]:
#Exploring data
#df.iloc[200:215] # few rows of dataframe
#df.iloc[:,42:57] # last few column of data frame 
#https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/
df.shape

(12771253, 58)

In [13]:
%time df_mod = df.copy() # create df_mod and use for experiments
df_mod.shape

Wall time: 6.28 s


(12771253, 58)

In [14]:
#Exploring data
#df_mod.iloc[200:215] # few rows of dataframe
#df_mod.iloc[:,35:50] # last few column of data frame 

In [15]:
#Change all nan in CANCELLATION_CODE column to letter E - E stands for "No Reason /Reason unknown"
%time display(df_mod.CANCELLATION_CODE.unique())
%time df_mod['CANCELLATION_CODE'].fillna('E', inplace=True)
%time display(df_mod.CANCELLATION_CODE.unique())

array([nan, 'A', 'C', 'B', 'D'], dtype=object)

Wall time: 982 ms
Wall time: 986 ms


array(['E', 'A', 'C', 'B', 'D'], dtype=object)

Wall time: 784 ms


#### CONVERT ALL ROWS WITH NAN THAT HAS A,B,C,D IN THEIR COLUMNS
- This will add benefits to analysis with cancellation code column later on
- This will conserve and reservation cancellation codes to be used later

In [16]:
%time df_mod[(df_mod['CANCELLATION_CODE']=='A')] = df_mod[(df_mod['CANCELLATION_CODE']=='A')].replace(to_replace = np.nan, value = 0)
%time df_mod[(df_mod['CANCELLATION_CODE']=='B')] = df_mod[(df_mod['CANCELLATION_CODE']=='B')].replace(to_replace = np.nan, value = 0)
%time df_mod[(df_mod['CANCELLATION_CODE']=='C')] = df_mod[(df_mod['CANCELLATION_CODE']=='C')].replace(to_replace = np.nan, value = 0)
%time df_mod[(df_mod['CANCELLATION_CODE']=='D')] = df_mod[(df_mod['CANCELLATION_CODE']=='D')].replace(to_replace = np.nan, value = 0) 
df_mod.shape

Wall time: 21.8 s
Wall time: 21.1 s
Wall time: 21.4 s
Wall time: 22.8 s


(12771253, 58)

In [17]:
#df_mod[(df_mod['CANCELLATION_CODE']=='B')]

## REASONS FOR DROPPING ROWS
- Samantha noticed the difference that all negative numbers for depature delays had NAN in other columns
- After futher investigation, description of Dep Delay is Difference in minutes between scheduled and actual departure time. Early departures show negative numbers.
### No need to take early depatures into consideration since investigation is being done on delay hence  the dropping of rows which  also eliminated the NANs
- this reduced the dataset from (12771253, 58) to (2660084, 58)
    - Still a healthy size dataset

In [18]:
#Drop rows with NAN values in data frame
#%time df_mod = df.copy() # create df_mod and use for experiments
####
#%time df_mod = df_mod.apply (pd.to_numeric, errors='coerce')
#%time df_mod = df_mod.dropna(inplace=True)
%time df_mod = df_mod.dropna()
%time df_mod = df_mod.reset_index(drop=True)

%time display(df_mod.shape)
#%time display(df_mod.head(7))

Wall time: 31 s
Wall time: 1.9 s


(2660084, 58)

Wall time: 2.99 ms


In [19]:
%time round(df_mod.isnull().mean()*100,2) 

Wall time: 3.94 s


YEAR                    0
QUARTER                 0
MONTH                   0
DAY_OF_MONTH            0
DAY_OF_WEEK             0
FL_DATE                 0
OP_UNIQUE_CARRIER       0
OP_CARRIER_AIRLINE_ID   0
OP_CARRIER              0
OP_CARRIER_FL_NUM       0
ORIGIN_AIRPORT_ID       0
ORIGIN_AIRPORT_SEQ_ID   0
ORIGIN_CITY_MARKET_ID   0
ORIGIN                  0
ORIGIN_CITY_NAME        0
ORIGIN_STATE_ABR        0
ORIGIN_STATE_NM         0
DEST_AIRPORT_ID         0
DEST_AIRPORT_SEQ_ID     0
DEST_CITY_MARKET_ID     0
DEST                    0
DEST_CITY_NAME          0
DEST_STATE_ABR          0
DEST_STATE_NM           0
CRS_DEP_TIME            0
DEP_TIME                0
DEP_DELAY               0
DEP_DELAY_NEW           0
DEP_DEL15               0
DEP_DELAY_GROUP         0
DEP_TIME_BLK            0
TAXI_OUT                0
WHEELS_OFF              0
WHEELS_ON               0
TAXI_IN                 0
CRS_ARR_TIME            0
ARR_TIME                0
ARR_DELAY               0
ARR_DELAY_NE

In [20]:
display(df_mod.columns)
display(df_mod.CANCELLATION_CODE.unique())

Index(['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER',
       'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID',
       'ORIGIN_CITY_MARKET_ID', 'ORIGIN', 'ORIGIN_CITY_NAME',
       'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM', 'DEST_AIRPORT_ID',
       'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR', 'DEST_STATE_NM', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15', 'DEP_DELAY_GROUP',
       'DEP_TIME_BLK', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN',
       'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'ARR_DELAY_NEW', 'ARR_DEL15',
       'ARR_DELAY_GROUP', 'ARR_TIME_BLK', 'CANCELLED', 'CANCELLATION_CODE',
       'DIVERTED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME',
       'FLIGHTS', 'DISTANCE', 'DISTANCE_GROUP', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_D

array(['E', 'A', 'C', 'B', 'D'], dtype=object)

In [21]:
#import carrier name file and merge onto df_mod to have carrier name description attached
#OR a map can be created to point all the carrier abbrv to the carrier names 
CN = pd.read_pickle('CN.pkl')
display(CN.shape)
display(CN.info())
display(CN.head(3))
'''
#CN1 = CN.drop_duplicates()
#importing excel version of CN and dropping duplicate rows
CN1.shape
df_mod.shape
#CN1.reset_index(inplace = True)
#CN1 = CN1.drop(['index'], axis=1)
#https://stackoverflow.com/questions/39019591/duplicated-rows-when-merging-dataframes-in-python
'''

(18, 2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
OP_CARRIER      18 non-null object
carrier_name    18 non-null object
dtypes: object(2)
memory usage: 368.0+ bytes


None

,OP_CARRIER,carrier_name
0,MQ,Envoy Air
1,NK,Spirit Air Lines
2,OH,PSA Airlines Inc.


"\n#CN1 = CN.drop_duplicates()\n#importing excel version of CN and dropping duplicate rows\nCN1.shape\ndf_mod.shape\n#CN1.reset_index(inplace = True)\n#CN1 = CN1.drop(['index'], axis=1)\n#https://stackoverflow.com/questions/39019591/duplicated-rows-when-merging-dataframes-in-python\n"

In [22]:
%time df_mod1 =  pd.merge(df_mod, CN, on='OP_CARRIER')
display(df_mod.shape)
display(df_mod1.shape)
#https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html

Wall time: 18.7 s


(2660084, 58)

(2660084, 59)

In [23]:
#import airport codes and merge to dataframe
A1 = pd.read_csv('L_AIRPORT_ID.csv')
display(A1.shape)
display(A1.info())
display(A1.head(3))

(6527, 2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6527 entries, 0 to 6526
Data columns (total 2 columns):
Code           6527 non-null int64
Description    6527 non-null object
dtypes: int64(1), object(1)
memory usage: 102.1+ KB


None

,Code,Description
0,10001,"Afognak Lake, AK: Afognak Lake Airport"
1,10003,"Granite Mountain, AK: Bear Creek Mining Strip"
2,10004,"Lik, AK: Lik Mining Camp"


In [24]:
#Rename column for merging to dataframe
A1 = A1.rename({'Code': 'ORIGIN_AIRPORT_ID'}, axis=1) 
#ORIGIN_AIRPORT_ID

In [25]:
display(df_mod1.shape)
%time df_mod1 =  pd.merge(df_mod1, A1, on='ORIGIN_AIRPORT_ID')
display(df_mod1.shape)

(2660084, 59)

Wall time: 5.32 s


(2660084, 60)

In [26]:
#Rename airport description column in dataframe
df_mod1 = df_mod1.rename({'Description': 'ORIGIN_AIRPORT_DESC'}, axis=1) 

In [27]:
display(df_mod1.head(3))

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,OP_CARRIER_FL_NUM,...,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Monthd,MY,carrier_name,ORIGIN_AIRPORT_DESC
0,2018,4,10,1,1,2018-10-01,WN,19393,WN,4025,...,4,2,0,1,0,13,Oct,Oct_2018,Southwest Airlines Co.,"Albuquerque, NM: Albuquerque International Sun..."
1,2018,4,10,1,1,2018-10-01,WN,19393,WN,5757,...,3,0,0,0,0,232,Oct,Oct_2018,Southwest Airlines Co.,"Albuquerque, NM: Albuquerque International Sun..."
2,2018,4,10,1,1,2018-10-01,WN,19393,WN,538,...,5,0,0,0,0,34,Oct,Oct_2018,Southwest Airlines Co.,"Albuquerque, NM: Albuquerque International Sun..."


In [28]:
display(df_mod1.OP_CARRIER.unique())
display(df_mod1.carrier_name.unique())

array(['WN', 'AA', 'AS', 'B6', 'DL', 'F9', 'G4', 'MQ', 'UA', 'YV', 'YX',
       'EV', 'OO', 'NK', 'OH', '9E', 'VX', 'HA'], dtype=object)

array(['Southwest Airlines Co.', 'American Airlines Inc.',
       'Alaska Airlines Inc.', 'JetBlue Airways', 'Delta Air Lines Inc.',
       'Frontier Airlines Inc.', 'Allegiant Air', 'Envoy Air',
       'United Air Lines Inc.', 'Mesa Airlines Inc.', 'Republic Airline',
       'ExpressJet Airlines Inc.', 'SkyWest Airlines Inc.',
       'Spirit Air Lines', 'PSA Airlines Inc.', 'Endeavor Air Inc.',
       'Virgin America', 'Hawaiian Airlines Inc.'], dtype=object)

In [29]:
display(df_mod1.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2660084 entries, 0 to 2660083
Data columns (total 60 columns):
YEAR                     int64
QUARTER                  int64
MONTH                    int64
DAY_OF_MONTH             int64
DAY_OF_WEEK              int64
FL_DATE                  object
OP_UNIQUE_CARRIER        object
OP_CARRIER_AIRLINE_ID    int64
OP_CARRIER               object
OP_CARRIER_FL_NUM        int64
ORIGIN_AIRPORT_ID        int64
ORIGIN_AIRPORT_SEQ_ID    int64
ORIGIN_CITY_MARKET_ID    int64
ORIGIN                   object
ORIGIN_CITY_NAME         object
ORIGIN_STATE_ABR         object
ORIGIN_STATE_NM          object
DEST_AIRPORT_ID          int64
DEST_AIRPORT_SEQ_ID      int64
DEST_CITY_MARKET_ID      int64
DEST                     object
DEST_CITY_NAME           object
DEST_STATE_ABR           object
DEST_STATE_NM            object
CRS_DEP_TIME             int64
DEP_TIME                 float64
DEP_DELAY                float64
DEP_DELAY_NEW            float64
DE

None

In [30]:
#Renaming dataframe
%time flights = df_mod1.copy()

Wall time: 1.45 s


In [31]:
#USE IMPORTED AIRPORT CODE AND ADD DESTINATION
#CHANGE NAME OF COLUMN TO DESTINATION - ORIGIN CODE DATASET IS SAME AS DEST CODE

In [32]:
#Rename column for merging to dataframe
A2 = A1.rename({'ORIGIN_AIRPORT_ID': 'DEST_AIRPORT_ID'}, axis=1) 
#DEST_AIRPORT_ID
display(A2.head(3))

,DEST_AIRPORT_ID,Description
0,10001,"Afognak Lake, AK: Afognak Lake Airport"
1,10003,"Granite Mountain, AK: Bear Creek Mining Strip"
2,10004,"Lik, AK: Lik Mining Camp"


In [33]:
display(flights.shape)
%time flights =  pd.merge(flights, A2, on='DEST_AIRPORT_ID')
display(flights.shape)

(2660084, 60)

Wall time: 33.7 s


(2660084, 61)

In [34]:
#Rename airport description column in dataframe
flights = flights.rename({'Description': 'DEST_AIRPORT_DESC'}, axis=1) 

In [35]:
#split airpot out the state column from destination airport description
%time split = flights['DEST_AIRPORT_DESC'].str.split(":", n = 1, expand = True) 
%time flights = pd.concat([flights, split], axis = 1)
#drop the column for 'DEST_AIRPORT_DESC''
%time flights = flights.drop(columns = ['DEST_AIRPORT_DESC'])
%time flights.rename(columns={0: 'City_State', 1:'DEST_AirportName'}, inplace=True)
%time flights = flights.drop(columns = ['City_State'])

Wall time: 9.84 s
Wall time: 2.64 s
Wall time: 7.05 s
Wall time: 1.99 s
Wall time: 2.23 s


In [36]:
#Stripping all trailing empty spaces in a column of dataframe
flights['DEST_AirportName'] = flights['DEST_AirportName'].map(lambda x: x.strip())

In [37]:
df_latlon = pd.read_csv('Dataset_AirportList_Main.csv')
display(df_latlon.shape)
display(df_latlon.head(3))

(362, 7)

,Unnamed: 0,City,State,AirportName,latitude_deg,longitude_deg,elevation_ft
0,352,Adak Island,AK,Adak,52,-177,18
1,299,Bethel,AK,Bethel Airport,61,-162,126
2,328,Deadhorse,AK,Deadhorse Airport,70,-148,65


In [38]:
#Dropping  columns not needed and Renaming columns 
%time df_latlon= df_latlon.drop(columns = ['Unnamed: 0', 'City','State'])
%time df_latlon.rename(columns={'AirportName': 'DEST_AirportName'},inplace=True)
#Stripping all trailing empty spaces in a needed columns of dataframe
df_latlon['DEST_AirportName'] = df_latlon['DEST_AirportName'].map(lambda x: x.strip())
display(df_latlon.head(3))

Wall time: 2 ms
Wall time: 1.53 ms


,DEST_AirportName,latitude_deg,longitude_deg,elevation_ft
0,Adak,52,-177,18
1,Bethel Airport,61,-162,126
2,Deadhorse Airport,70,-148,65


In [39]:
#MERGE DF_LATLON TO FLIGHTS
#pd.options.display.float_format = '{:.0f}'.format
pd.options.display.float_format = '{:,.4f}'.format
%time flights =  pd.merge(flights, df_latlon, on='DEST_AirportName')
%time flights.rename(columns={'latitude_deg': 'DEST_latitude', 'longitude_deg': 'DEST_longitude', 'elevation_ft': 'DEST_elevation_ft'},inplace=True)
display(flights.shape)
display(flights.head(2))

Wall time: 22.6 s
Wall time: 6.4 s


(2661994, 64)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,OP_CARRIER_FL_NUM,...,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Monthd,MY,carrier_name,ORIGIN_AIRPORT_DESC,DEST_AirportName,DEST_latitude,DEST_longitude,DEST_elevation_ft
0,2018,4,10,1,1,2018-10-01,WN,19393,WN,4025,...,0.0000,13.0000,Oct,Oct_2018,Southwest Airlines Co.,"Albuquerque, NM: Albuquerque International Sun...",William P Hobby,29.6454,-95.2789,46
1,2018,4,10,26,5,2018-10-26,WN,19393,WN,530,...,0.0000,0.0000,Oct,Oct_2018,Southwest Airlines Co.,"Albuquerque, NM: Albuquerque International Sun...",William P Hobby,29.6454,-95.2789,46


In [40]:
#split airport out the state column from ORIGIN airport description
%time split = flights['ORIGIN_AIRPORT_DESC'].str.split(":", n = 1, expand = True) 
%time flights = pd.concat([flights, split], axis = 1)
#drop the column for 'DEST_AIRPORT_DESC''
%time flights = flights.drop(columns = ['ORIGIN_AIRPORT_DESC'])
%time flights.rename(columns={0: 'City_State', 1:'ORIGIN_AirportName'}, inplace=True)
%time flights = flights.drop(columns = ['City_State'])
#Stripping all trailing empty spaces in a column of dataframe
flights['ORIGIN_AirportName'] = flights['ORIGIN_AirportName'].map(lambda x: x.strip())

Wall time: 11.8 s
Wall time: 2.7 s
Wall time: 6.87 s
Wall time: 2.67 s
Wall time: 2.64 s


In [41]:
#RENAME DEST_AIRPORT COLUMNS TO ORIGIN
%time df_latlon.rename(columns={'DEST_AirportName': 'ORIGIN_AirportName'},inplace=True)
display(df_latlon.head(3))

Wall time: 998 µs


,ORIGIN_AirportName,latitude_deg,longitude_deg,elevation_ft
0,Adak,51.8780,-176.6460,18
1,Bethel Airport,60.7798,-161.8380,126
2,Deadhorse Airport,70.1947,-148.4650,65


In [42]:
#MERGE DF_LATLON TO FLIGHTS FOR ORIGINS
pd.options.display.float_format = '{:,.4f}'.format
display(flights.shape)
%time flights =  pd.merge(flights, df_latlon, on='ORIGIN_AirportName')
%time flights.rename(columns={'latitude_deg': 'ORIGIN_latitude', 'longitude_deg': 'ORIGIN_longitude', 'elevation_ft': 'ORIGIN_elevation_ft'},inplace=True)
display(flights.shape)
display(flights.head(2))

(2661994, 64)

Wall time: 26.4 s
Wall time: 6.8 s


(2663675, 67)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,OP_CARRIER_FL_NUM,...,MY,carrier_name,DEST_AirportName,DEST_latitude,DEST_longitude,DEST_elevation_ft,ORIGIN_AirportName,ORIGIN_latitude,ORIGIN_longitude,ORIGIN_elevation_ft
0,2018,4,10,1,1,2018-10-01,WN,19393,WN,4025,...,Oct_2018,Southwest Airlines Co.,William P Hobby,29.6454,-95.2789,46,Albuquerque International Sunport,35.0402,-106.6090,5355
1,2018,4,10,26,5,2018-10-26,WN,19393,WN,530,...,Oct_2018,Southwest Airlines Co.,William P Hobby,29.6454,-95.2789,46,Albuquerque International Sunport,35.0402,-106.6090,5355


In [43]:
flights.columns

Index(['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER',
       'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID',
       'ORIGIN_CITY_MARKET_ID', 'ORIGIN', 'ORIGIN_CITY_NAME',
       'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM', 'DEST_AIRPORT_ID',
       'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR', 'DEST_STATE_NM', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15', 'DEP_DELAY_GROUP',
       'DEP_TIME_BLK', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN',
       'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'ARR_DELAY_NEW', 'ARR_DEL15',
       'ARR_DELAY_GROUP', 'ARR_TIME_BLK', 'CANCELLED', 'CANCELLATION_CODE',
       'DIVERTED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME',
       'FLIGHTS', 'DISTANCE', 'DISTANCE_GROUP', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_D

In [44]:
flights.to_pickle('flights.pkl') # saving modified dataframe
#%time flights = pd.read_pickle('flights.pkl') #read and load saved dataframe to continued analysis